### Working page leafthrough

In [ ]:
import gradio as gr

def paginate(page, max_page, page_change):
    new_page = page + page_change
    new_page = max(1, new_page)  # Ensures page is not less than 1
    new_page = min(new_page, max_page)  # Ensures page does not exceed max_page
    return new_page

def paginate_go(page, max_page):
    try:
        page = int(page)
    except ValueError:
        print(f'Invalid page number: {page}')
        return None
    return paginate(page, max_page, 0)
def create_gradio_app():
    with gr.Blocks() as app:
        with gr.Row():
            page_input = gr.Number(label="Page Number", value=1)
            max_page = gr.Number(label="Max Page", value=10)  # Assuming max 10 pages for testing
            go_button = gr.Button("Go to Page")
            next_button = gr.Button("Next")
            prev_button = gr.Button("Previous")
            current_page_label = gr.Label("Current Page: 1")

        def update_page_label(page):
            current_page_label.update(f"Current Page: {page}")

        go_button.click(fn=paginate_go, inputs=[page_input, max_page], outputs=page_input)
        next_button.click(fn=lambda x, y: paginate(x, y, 1), inputs=[page_input, max_page], outputs=page_input)
        prev_button.click(fn=lambda x, y: paginate(x, y, -1), inputs=[page_input, max_page], outputs=page_input)
        page_input.change(update_page_label)

    return app

app = create_gradio_app()
app.launch()

In [45]:
import gradio as gr
import math

IMAGES_TO_SHOW = 2  # Number of images to show per page
loaded_images = []  # Global variable to store the loaded images

def paginate(page, max_page, page_change):
    new_page = page + page_change
    new_page = max(1, new_page)
    new_page = min(new_page, max_page)
    return new_page

def get_images_for_page(all_images, page):
    start_index = (page - 1) * IMAGES_TO_SHOW
    end_index = start_index + IMAGES_TO_SHOW
    return all_images[start_index:end_index]

def update_gallery(page, max_page):
    global loaded_images
    displayed_images = get_images_for_page(loaded_images, page)
    return displayed_images, page, f"Current Page: {page}"

def handle_pagination(page, max_page, delta):
    new_page = paginate(page, max_page, delta)
    return update_gallery(new_page, max_page)

def handle_image_load(images):
    global loaded_images
    loaded_images = images
    max_page = math.ceil(len(loaded_images) / IMAGES_TO_SHOW)
    return update_gallery(1, max_page)  # Display first page of images

def create_gradio_app():
    with gr.Blocks() as app:
        with gr.Row():
            image_loader = gr.File(label="Load Images", file_count='multiple')
            image_gallery = gr.Gallery(label="Image Gallery")

        with gr.Row():
            page_input = gr.Number(label="Page Number", value=1, visible=True)
            current_page_label = gr.Label("Current Page: 1")
            go_button = gr.Button("Go to Page")
            prev_button = gr.Button("Previous")
            next_button = gr.Button("Next")

        image_loader.change(fn=handle_image_load, inputs=[image_loader], outputs=[image_gallery, page_input, current_page_label])
        next_button.click(fn=lambda page: handle_pagination(page, math.ceil(len(loaded_images) / IMAGES_TO_SHOW), 1), inputs=[page_input], outputs=[image_gallery, page_input, current_page_label])
        prev_button.click(fn=lambda page: handle_pagination(page, math.ceil(len(loaded_images) / IMAGES_TO_SHOW), -1), inputs=[page_input], outputs=[image_gallery, page_input, current_page_label])
        go_button.click(fn=lambda page: update_gallery(page, math.ceil(len(loaded_images) / IMAGES_TO_SHOW)), inputs=[page_input], outputs=[image_gallery, page_input, current_page_label])

    return app

app = create_gradio_app()
app.launch()


Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.
